Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [17]:
import pandas as pd
import numpy as np

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [20]:
def wrangle (df):
    # Prevent warning
    df = df.copy(deep=True)

    # Date time conversion
    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['date_recorded_year'] = df['date_recorded'].dt.year
    df['date_recorded_month'] = df['date_recorded'].dt.month
    df['date_recorded_day'] = df['date_recorded'].dt.day
    
    # Deal with outliers
#     df['population'] = (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
#        (df['latitude'] < np.percentile(df['latitude'], 99.95)) 
#     df['num_private'] = might as well be tossed if all non-zeros are trimmed
#     df['amount_tsh'] = 
    
    # Fix zeros
    def fix_zeros (col):
        df[col][df[col]==0] = np.NaN
    
    fix_zeros('longitude')
    fix_zeros('latitude')
    fix_zeros('construction_year')
    fix_zeros('gps_height')
    
    # Engineer feature
    df['Age'] = df['date_recorded_year'] - df['construction_year']
    df['region_district_comb'] = df['region_code']*100 + df['district_code']
    
    # select dropped cols and return
    drops = ['date_recorded','id','recorded_by', 'wpt_name']
    return df.drop(columns=drops)

train_wrangled = wrangle(train)
test_wrangled = wrangle(test)

C:\Users\Owner\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [13]:
pd.set_option('display.max_columns',40)

In [21]:
test_wrangled.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,...,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,date_recorded_year,date_recorded_month,date_recorded_day,Age,region_district_comb
0,0.0,Dmdd,1996.0,DMDD,35.290799,-4.059696,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,Parastatal,NaN,True,2012.0,...,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,2013,2,4,1.0,2103
1,0.0,Government Of Tanzania,1569.0,DWE,36.656709,-3.309214,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,VWC,TPRI pipe line,True,2000.0,...,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,2013,2,4,13.0,202
2,0.0,NaN,1567.0,NaN,34.767863,-5.004344,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,VWC,P,NaN,2010.0,...,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,2013,2,1,3.0,1302
3,0.0,Finn Water,267.0,FINN WATER,38.058046,-9.418672,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,VWC,NaN,True,1987.0,...,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other,2013,1,22,26.0,8043
4,500.0,Bruder,1260.0,BRUDER,35.006123,-10.950412,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,Water Board,BRUDER,True,2000.0,...,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2013,3,27,13.0,1003


In [23]:
target = 'status_group'
features = train_wrangled.columns.drop([target])

X_train = train_wrangled[features]
y_train = train_wrangled[target]

In [25]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import validation_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [27]:
pipeline = make_pipeline(ce.OrdinalEncoder(),
                        SimpleImputer(),
                        RandomForestClassifier(n_estimators=50,
                                              max_depth=20,
                                              n_jobs=-1,
                                              ccp_alpha=0.0))

param_distributions = {
    'simpleimputer__strategy':['mean','median','most_frequent'],
    'randomforestclassifier__max_depth':[5, 10, 15, 20, None],
    'randomforestclassifier__ccp_alpha':[0,0.3,0.6,0.9]
}

search = RandomizedSearchCV(pipeline,
                           param_distributions=param_distributions,
                           cv=5)

In [28]:
search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [30]:
search.best_score_

0.8106397306397307

In [33]:
predictions_223 = search.best_estimator_.predict(test_wrangled)

In [34]:
predictions_223 = pd.DataFrame(data={'id':sample_submission['id'],'status_group':predictions_223})

In [35]:
predictions_223.to_csv('predictions_223.csv',index=False)

In [9]:
(test.id != sample_submission.id).any()

False